# Library

In [1]:
from tqdm import tqdm
import numpy as np
import re

In [2]:
import pandas as pd

In [3]:
import ray
ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}})

2022-11-28 12:56:27,268	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.13
Ray version:,2.0.1
Dashboard:,http://127.0.0.1:8265


## data load

In [4]:
col = pd.read_table('data/컬럼명.txt', sep=',')
col

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,배기량,승차인원,차량길이,차량너비,차량높이,원산지명,삭제YN,제원승인일자,구동형식,변속기종류


In [5]:
col.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명', '차종_제원정보',
       '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력',
       '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류'],
      dtype='object')

In [6]:
# 23.4s
df = pd.read_table('data/220819_22.06월기준_4등급.txt', encoding='cp949', header=40, names=col.columns)
df.info()

C:\Users\kbj\AppData\Local\Temp\ipykernel_18336\2626983995.py:2: DtypeWarning: Columns (14,17,29,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_table('data/220819_22.06월기준_4등급.txt', encoding='cp949', header=40, names=col.columns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528586 entries, 0 to 1528585
Data columns (total 40 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   차량관리번호       1528586 non-null  object 
 1   차량등록번호       1528586 non-null  object 
 2   본거지법정동코드     1528586 non-null  int64  
 3   소유자구분        1528586 non-null  object 
 4   차종_등록정보      1528584 non-null  object 
 5   용도           1528586 non-null  object 
 6   차대번호         1528586 non-null  object 
 7   최초등록일자       1528586 non-null  int64  
 8   차량연식         1528586 non-null  int64  
 9   차량제작일자       1358925 non-null  float64
 10  검사유효일        1528574 non-null  float64
 11  제원관리번호       1528586 non-null  object 
 12  배출가스인증번호     1405585 non-null  object 
 13  배출가스등급       1528586 non-null  int64  
 14  저공해조치종류      195 non-null      object 
 15  저공해조치구조변경YN  1528586 non-null  object 
 16  차량말소YN       1528586 non-null  object 
 17  취득일자         1435514 non-null  object 
 18  취득

In [7]:
df.head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,배기량,승차인원,차량길이,차량너비,차량높이,원산지명,삭제YN,제원승인일자,구동형식,변속기종류
0,KMFZCY7JBCU784032-01,97구4490,1144010700,법인,화물,자가용,KMFZCY7JBCU784032,20110817,2012,20110811.0,...,2497.0,6,5120.0,1740.0,1965.0,대한민국,N,20110524,NaN,자동
1,KMHJN81VP7U578447-01,11너2154,2650010100,주민,승용,자가용,KMHJN81VP7U578447,20070102,2007,20061104.0,...,1991.0,5,4325.0,1800.0,1680.0,대한민국,N,20060621,NaN,자동
2,KPACE4AB1AP070600-01,96어5714,4729025024,주민,화물,자가용,KPACE4AB1AP070600,20091221,2010,20091210.0,...,1998.0,5,4965.0,1900.0,1755.0,대한민국,N,20090513,NaN,자동
3,KMHNU81WP7U009391-01,11가1154,4117110100,사업자,승용,자가용,KMHNU81WP7U009391,20070212,2007,20070207.0,...,2959.0,7,4840.0,1970.0,1805.0,대한민국,N,20060821,NaN,자동
4,KMHDT41TP9U638425-01,22보7099,4711331026,주민,승용,자가용,KMHDT41TP9U638425,20081128,2009,20081123.0,...,1582.0,5,4505.0,1775.0,1480.0,대한민국,N,20080729,NaN,수동


### 파일로 출력

In [8]:
# # 34.6s
# df.to_csv('data/220819_22.06월기준_4등급.csv', encoding='utf-8')

### 0\. [e]차량관리번호
- 오류
    - null 값 : 0 개
    - 영문과 숫자가 아닌 값 : 7,038 개

In [9]:
df.iloc[:, 0].isnull().sum()

0

In [10]:
df.iloc[:, 0].value_counts()

KMFZCY7JBCU784032-01    1
KMHSH81VP9U467470-01    1
KNAGA2263NA273801-01    1
KNCSJX74BBK577862-01    1
KNADG414AAS597155-01    1
                       ..
KNAFA2223PA131127-01    1
KPACA4AB1BP099521-01    1
KNCSJZ74BBK571321-01    1
KNAGA2223MA251640-01    1
KNME5C2MM8P008790-01    1
Name: 차량관리번호, Length: 1528586, dtype: int64

In [11]:
count = 0
p0 = re.compile('\w+[-]\d+')
idx0 = []
for i, one in tqdm(enumerate(df.iloc[:, 0].to_numpy())):
    if not p0.search(one):
        idx0.append(i)
        count += 1
count

1528586it [00:01, 1059478.27it/s]


7038

In [12]:
idx0[:5]

[396, 586, 700, 821, 1227]

In [13]:
df[['차량관리번호', '차량등록번호']].reindex(idx0)

,차량관리번호,차량등록번호
396,1199서울4브2371,서울4브2371
586,4599전북1우1923,전북1우1923
700,1199서울1즈5001,서울1즈5001
821,2799대구4수5818,대구4수5818
1227,4199경기59너9481,경기59너9481
...,...,...
1527680,1199서울52마5593,서울52마5593
1527890,4199경기3노8469,경기3노8469
1528062,3099대전1므5676,대전1므5676
1528409,1199서울2두4957,서울2두4957


### 1\. [e]차량등록번호
- 오류
    - null 값 : 0 개
    - [한글][숫자][한글][숫자] 형식에 맞지 않는 값 : 1,211,007 개
    - [숫자][한글][숫자] 형식에 맞지 않는 값 : 0 개

In [14]:
df.iloc[:, 1].isnull().sum()

0

In [15]:
df.iloc[:, 1].value_counts()

97구4490      1
27나1278      1
인천1더9940     1
83두1780      1
12소2340      1
            ..
서울1러2561     1
95마1880      1
87마2994      1
경기45마7935    1
55고3109      1
Name: 차량등록번호, Length: 1528586, dtype: int64

In [16]:
count = 0
p11 = re.compile('\d+[가-힣]+')
idx11 = []
for i, one in tqdm(enumerate(df.iloc[:, 1].to_numpy())):
    if not p11.search(one):
        idx11.append(i)
        count += 1
count

1528586it [00:01, 1123053.94it/s]


0

In [17]:
count = 0
p12 = re.compile('[가-힣]+\d+[가-힣]+\d+')
idx12 = []
for i, one in tqdm(enumerate(df.iloc[:, 1].to_numpy())):
    if not p12.search(one):
        idx12.append(i)
        count += 1
count

1528586it [00:01, 884508.23it/s]


1211007

In [18]:
df.iloc[:, 1].reindex(idx12)[:5]

C:\Users\kbj\AppData\Local\Temp\ipykernel_18336\1435103437.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df.iloc[:, 1].reindex(idx12)[:5]


0    97구4490
1    11너2154
2    96어5714
3    11가1154
4    22보7099
Name: 차량등록번호, dtype: object

In [19]:
df.iloc[:, [0, 1, 6]].reindex(idx12)[:5]

,차량관리번호,차량등록번호,차대번호
0,KMFZCY7JBCU784032-01,97구4490,KMFZCY7JBCU784032
1,KMHJN81VP7U578447-01,11너2154,KMHJN81VP7U578447
2,KPACE4AB1AP070600-01,96어5714,KPACE4AB1AP070600
3,KMHNU81WP7U009391-01,11가1154,KMHNU81WP7U009391
4,KMHDT41TP9U638425-01,22보7099,KMHDT41TP9U638425


In [20]:
df.iloc[:, [0, 1, 6]].reindex(idx12)[-5:]

,차량관리번호,차량등록번호,차대번호
1528580,KNAJE55539K557097-01,22더4719,KNAJE55539K557097
1528581,KMHJN81VP6U438041-01,06다4703,KMHJN81VP6U438041
1528582,KNAKH815BAK669764-01,17무6646,KNAKH815BAK669764
1528583,KMCLK19KP8C035195-01,96부7399,KMCLK18KP8C035195
1528585,KNME5C2MM8P008790-01,55고3109,KNME5C2MM8P008790


In [21]:
count = 0
p13 = re.compile('\d+[가-힣]+\d+')
idx13 = []
for i, one in tqdm(enumerate(df.iloc[:, 1].to_numpy())):
    if not p13.search(one):
        idx13.append(i)
        count += 1
count

1528586it [00:01, 1057805.76it/s]


0

### 2\. 본거지법정동코드
- 오류
    - null 값 : 0 개
    - 숫자가 아닌 값 : 0 개
    - 길이가 10이 아닌 값 : 0 개

In [22]:
df.iloc[:, 2].isnull().sum()

0

In [23]:
df.iloc[:, 2].value_counts()

4139013200    5442
1150010300    4246
1162010200    3960
2817710500    3936
1135010500    3513
              ... 
4479039025       1
4575039000       1
4119710400       1
4163055000       1
4371025030       1
Name: 본거지법정동코드, Length: 20119, dtype: int64

In [24]:
# 숫자가 아닌 값
count = 0
p2 = re.compile('\D+')
idx2 = []
for i, one in tqdm(enumerate(df.iloc[:, 2].to_numpy())):
    if p2.search(str(one)):
        idx2.append(i)
        count += 1
count

1528586it [00:02, 562088.29it/s]


0

In [25]:
p2.search('2b안녕하세요1a')

<re.Match object; span=(1, 7), match='b안녕하세요'>

In [26]:
count = 0
idx21 = []
for i, one in tqdm(enumerate(df.iloc[:, 2].to_numpy())):
    if len(str(one)) != 10:
        idx21.append(i)
        count += 1
count

1528586it [00:01, 841358.37it/s]


0

### 3\. 소유자구분

In [27]:
df.iloc[:, 3].value_counts()

주민     1320642
법인      139402
사업자      42860
외국인      21568
기타        4114
Name: 소유자구분, dtype: int64

### 4\. 차종_등록정보

In [28]:
df.iloc[:, 4].value_counts()

승용    850752
화물    520825
승합    148852
특수      8155
Name: 차종_등록정보, dtype: int64

### 5\. 용도

In [29]:
df.iloc[:, 5].value_counts()

자가용    1481507
영업용      41362
관용        5717
Name: 용도, dtype: int64

### 6\. [e]차대번호
- 오류
    - 1\. null 값 : 0 개
    - 2\. 정상(17자리, 영문 포함) : 1,525,955 개
    - 3\. 17자리, 영문X : 1 개
    - 4\. 17자리X, 영문 : 57 개
    - 5\. 17자리X, 영문X : 2,573 개

In [30]:
df.iloc[:, 6].isnull().sum()

0

In [31]:
len('KMHCF21DPSU000012')

17

In [32]:
df.iloc[:, 6].value_counts()

KMHCF21DPSU000012    5
KLAVM69H1JB075787    2
KMHMF21FPPU018261    2
KMHVF21JPKU161128    2
KLY7T51BBNC011472    2
                    ..
KMFZCZ7JABU656518    1
KNAFA2223PA131127    1
KPACA4AB1BP099521    1
KNCSJZ74BBK571321    1
KNME5C2MM8P008790    1
Name: 차대번호, Length: 1528564, dtype: int64

In [33]:
df.shape

(1528586, 40)

In [34]:
# 17자리, 영문 -> 정상 : 1525955
count = 0
p61 = re.compile('[a-zA-Z]+')
idx61 = []
for i, one in tqdm(enumerate(df.iloc[:, 6].to_numpy())):
    if len(str(one)) == 17 and p61.search(str(one)):
        idx61.append(i)
        count += 1
count

1528586it [00:02, 556470.27it/s]


1525955

In [35]:
# 17자리, 영문X : 1
count = 0
p62 = re.compile('[a-zA-Z]+')
idx62 = []
for i, one in tqdm(enumerate(df.iloc[:, 6].to_numpy())):
    n = 0
    for j in str(one):
        if p62.search(j):
            n += 1
    if len(str(one)) == 17 and n == 0:
        idx62.append(i)
        count += 1
count

1528586it [00:12, 118895.74it/s]


1

In [36]:
df.iloc[:, 6].reindex(idx62)

708259    01901321371111111
Name: 차대번호, dtype: object

In [37]:
# 17자리 X, 영문 : 57
count = 0
p63 = re.compile('[a-zA-Z]+')
idx63 = []
for i, one in tqdm(enumerate(df.iloc[:, 6].to_numpy())):
    if len(str(one)) != 17 and p63.search(one):
        idx63.append(i)
        count += 1
count

1528586it [00:00, 1642538.76it/s]


57

In [38]:
df.iloc[:, 6].reindex(idx63)[:5]

C:\Users\kbj\AppData\Local\Temp\ipykernel_18336\137706125.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df.iloc[:, 6].reindex(idx63)[:5]


34589         SL68T032460
69588         RPS13330414
71114        CN-62-T-2213
91950         KPY30051283
96602    KNADA2322HS00898
Name: 차대번호, dtype: object

In [39]:
# 17자리 X, 영문X : 2573
count = 0
p64 = re.compile('[a-zA-Z]+')
idx64 = []
for i, one in tqdm(enumerate(df.iloc[:, 6].to_numpy())):
    n = 0
    for j in one:
        if p64.search(j):
            n += 1
    if len(str(one)) != 17 and n == 0:
        idx64.append(i)
        count += 1
count

1528586it [00:11, 127555.53it/s]


2573

In [40]:
df.iloc[:, 6].reindex(idx64)[:5]

C:\Users\kbj\AppData\Local\Temp\ipykernel_18336\420329397.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df.iloc[:, 6].reindex(idx64)[:5]


429       119811326107
692     27931130152305
1320      119111327184
1475      119811229792
1544      219211301625
Name: 차대번호, dtype: object

### 7\. [e]최초등록일자
- 오류
    - null 값 : 0 개
    - 숫자가 아닌 값 : 0 개
    - 길이가 8이 아닌 값 : 2 개

In [41]:
df.iloc[:, 7].value_counts()

20080102    4651
20110103    4609
20090102    4263
20080103    2707
20100104    2634
            ... 
20160127       1
20130617       1
20040716       1
20130712       1
20220614       1
Name: 최초등록일자, Length: 8446, dtype: int64

In [42]:
df.iloc[:, 7].isnull().sum()

0

In [43]:
count = 0
p7 = re.compile('\d+')
idx7 = []
for i, one in tqdm(enumerate(df.iloc[:, 7].to_numpy())):
    if not p7.search(str(one)):
        idx7.append(i)
        count += 1
count

1528586it [00:02, 677042.45it/s]


0

In [44]:
count = 0
idx71 = []
for i, one in tqdm(enumerate(df.iloc[:, 7].to_numpy())):
    if len(str(one)) != 8:
        idx71.append(i)
        count += 1
count

1528586it [00:01, 918736.69it/s]


2

In [45]:
df.iloc[:, 7].reindex(idx71)

110669       1993
489146    9960617
Name: 최초등록일자, dtype: int64

### 8\. 차량연식
- 오류
    - null 값 : 0 개
    - 길이 4 아닌 값 : 0 개
    - 숫자가 아닌 값 : 0 개

In [46]:
df.iloc[:, 8].value_counts()

2010    252711
2011    237575
2008    226868
2009    213544
2007    107029
2012     94187
1994     37783
1996     36266
1995     35395
1993     34281
2006     34010
1992     33927
1991     30328
1997     29988
1990     26704
1989     20689
1999     20030
2000     13327
1988     11854
1998     10236
2002      8971
2001      7836
1987      1278
2005      1068
2017       732
2018       483
2016       482
2015       463
2004       214
2003       169
2013        98
2014        13
2019        13
1986        12
1985         4
2020         3
1984         3
1979         3
1983         3
1978         1
1981         1
1980         1
1901         1
1968         1
1973         1
Name: 차량연식, dtype: int64

In [47]:
df.iloc[:, 8].isnull().sum()

0

In [48]:
count = 0
idx8 = []
for i, one in tqdm(enumerate(df.iloc[:, 8].to_numpy())):
    if len(str(one)) != 4:
        idx8.append(i)
        count += 1
count

1528586it [00:01, 904369.05it/s]


0

In [49]:
count = 0
p81 = re.compile('\d+')
idx81 = []
for i, one in tqdm(enumerate(df.iloc[:, 8].to_numpy())):
    if not p81.search(str(one)):
        idx81.append(i)
        count += 1
count

1528586it [00:02, 699257.89it/s]


0

### 9\. [e]차량제작일자
- 오류
    - null : 169,661 개
    - 8자리 아닌 값(null값 제외) : 4 개
    - 숫자가 아닌 값(null값 제외) : 0개

In [50]:
df.iloc[:, 9].isnull().sum()

169661

In [51]:
df.iloc[:, 9].value_counts()

20090629.0    1459
20090612.0    1444
20080219.0    1432
20091201.0    1424
20090702.0    1402
              ... 
20140628.0       1
20030407.0       1
20060219.0       1
20130128.0       1
20190923.0       1
Name: 차량제작일자, Length: 8299, dtype: int64

In [52]:
temp9 = df.iloc[:, 9].fillna(0).astype('int')
temp9[:5]

0    20110811
1    20061104
2    20091210
3    20070207
4    20081123
Name: 차량제작일자, dtype: int32

In [53]:
# 길이가 8이 아닌 값 : 169,665 개
count = 0
idx91 = []
for i, one in tqdm(enumerate(temp9.to_numpy())):
    if len(str(one)) != 8:
        idx91.append(i)
        count += 1
count

1528586it [00:01, 892363.52it/s]


169665

In [54]:
# null 값 : 169,661 개
count = 0
idx92 = []
for i, one in tqdm(enumerate(temp9.to_numpy())):
    if one == 0:
        idx92.append(i)
        count += 1
count

1528586it [00:00, 1665677.09it/s]


169661

In [55]:
# # 길이가 8이 아닌 값 중 null 값 제외 값 : 4 개
len(list(set(idx91)^set(idx92)))

4

In [56]:
temp9.reindex(list(set(idx91)^set(idx92)))

110669        1993
1202200    2100819
777510     9950407
489146     9960617
Name: 차량제작일자, dtype: int32

In [57]:
# 숫자가 아닌 값 : 0 개
count = 0
p93 = re.compile('\d+')
idx93 = []
for i, one in tqdm(enumerate(temp9.to_numpy())):
    if not p93.search(str(one)):
        idx93.append(i)
        count += 1
count

1528586it [00:02, 673383.50it/s]


0

### 10\. [e]검사유효일
- 오류
    - null : 12 개
    - 8자리 아닌 값 : 12 개
    - 숫자가 아닌 값(nan값 제외) : 0 개

In [58]:
df.iloc[:, 10].isnull().sum()

12

In [59]:
df.iloc[:, 10].value_counts()

20200103.0    3335
20200101.0    3304
20200102.0    3132
20210101.0    3017
20190831.0    2346
              ... 
21060605.0       1
19901201.0       1
19900208.0       1
19900510.0       1
20250908.0       1
Name: 검사유효일, Length: 12510, dtype: int64

In [60]:
temp10 = df.iloc[:, 10].fillna(0).astype('int')
temp10[:5]

0    20200508
1    20210101
2    20230130
3    20210211
4    20221231
Name: 검사유효일, dtype: int32

In [61]:
# 길이가 8이 아닌 값 : 12 개
count = 0
idx101 = []
for i, one in tqdm(enumerate(temp10.to_numpy())):
    if len(str(one)) != 8:
        idx101.append(i)
        count += 1
count

1528586it [00:01, 903427.18it/s]


12

In [62]:
# null 값 : 12 개
count = 0
idx102 = []
for i, one in tqdm(enumerate(temp10.to_numpy())):
    if one == 0:
        idx102.append(i)
        count += 1
count

1528586it [00:00, 1756496.47it/s]


12

In [63]:
# 길이가 8이 아닌 값 중 null 값 제외 값 : 0 개
    # => 길이가 8이 아닌 값은 전부 null 값
len(list(set(idx101)^set(idx102)))

0

In [64]:
# 숫자가 아닌 값 : 0 개
count = 0
p103 = re.compile('\d+')
idx103 = []
for i, one in tqdm(enumerate(temp10.to_numpy())):
    if not p103.search(str(one)):
        idx103.append(i)
        count += 1
count

1528586it [00:02, 659985.82it/s]


0

### 11\. [e]제원관리번호
- 오류
    - null : 0 개
    - 17자리X : 356,933 개
    - 17자리X, 숫자X : 30 개

In [65]:
df.iloc[:, 11].value_counts()

A0810006801591209    36231
A0110003400441206    22730
A0810006209233110    19414
A0810007402962210    16877
A0810006801661210    16186
                     ...  
B2F10014200003307        1
00032701002631207        1
30290100000046           1
20465900000000           1
A0810005209353308        1
Name: 제원관리번호, Length: 15315, dtype: int64

In [66]:
df.iloc[:, 11].isnull().sum()

0

In [67]:
# 17자리X : 356933
count = 0
idx111 = []
for i, one in tqdm(enumerate(df.iloc[:, 11].to_numpy())):
    if len(str(one)) != 17:
        idx111.append(i)
        count += 1
count

1528586it [00:01, 1460306.81it/s]


356933

In [68]:
# 17자리X, 숫자이외 값 : 30
count = 0
p112 = re.compile('[a-zA-Z가-힣]+')
idx112 = []
for i, one in tqdm(enumerate(df.iloc[:, 11].to_numpy())):
    if len(str(one)) != 17 and p112.search(one):
        idx112.append(i)
        count += 1
count

1528586it [00:01, 1321963.35it/s]


30

### 12\. [e]배출가스인증번호
- 오류
    - null : 123,001 개
    - 정상([숫자문자]-[숫자문자]-[숫자문자]-[숫자문자], 12자리) : 1,152,680 개
    - 오류 : 375,906 개
        - null 값 제외 : 252,905 개

In [69]:
df.iloc[:, 12].value_counts()

7MY-HD-14-68      268972
8MY-KM-13-19       93969
6MY-KM-14-61       76371
9MY-KM-14-41       67493
7MY-HD-14-46       60364
                   ...  
8MY-WI-0M-24           1
AMY-PPE-14-225         1
8MY-SN-0M-24           1
7MY-SK-24-03           1
AMY-PPE-14-800         1
Name: 배출가스인증번호, Length: 304, dtype: int64

In [70]:
df.iloc[:, 12].isnull().sum()

123001

In [71]:
# 정상 : 1152680
count = 0
p121 = re.compile('[a-zA-Z0-9]{3}[-][a-zA-Z]{2}[-][0-9]{2}[-][0-9]{2}')
idx121 = []
for i, one in tqdm(enumerate(df.iloc[:, 12].to_numpy())):
    if p121.search(str(one)):
        idx121.append(i)
        count += 1
count

1528586it [00:01, 938462.23it/s]


1152680

In [72]:
# 오류값 : 375906
count = 0
idx122 = []
for i, one in tqdm(enumerate(df.iloc[:, 12].to_numpy())):
    if not p121.search(str(one)):
        idx122.append(i)
        count += 1
count

1528586it [00:01, 999770.36it/s] 


375906

In [73]:
df.iloc[:, 12].reindex(idx122)[:5]

C:\Users\kbj\AppData\Local\Temp\ipykernel_18336\4092535168.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df.iloc[:, 12].reindex(idx122)[:5]


5     VMY-KM-38
21    RMY-HD-20
28    KMY-HD-03
29          NaN
36          NaN
Name: 배출가스인증번호, dtype: object

In [89]:
df[df['배출가스인증번호'].isnull() == True]['배출가스인증번호'].index

Int64Index([     29,      36,      42,      63,      67,      92,      98,
                108,     128,     147,
            ...
            1528534, 1528535, 1528536, 1528538, 1528539, 1528540, 1528548,
            1528561, 1528583, 1528584],
           dtype='int64', length=123001)

In [90]:
idx123 = df[df['배출가스인증번호'].isnull() == True]['배출가스인증번호'].index
len(idx123)

123001

In [91]:
df.iloc[:, 12].reindex(idx123)[:5]

C:\Users\kbj\AppData\Local\Temp\ipykernel_18336\495780953.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df.iloc[:, 12].reindex(idx123)[:5]


29    NaN
36    NaN
42    NaN
63    NaN
67    NaN
Name: 배출가스인증번호, dtype: object

In [92]:
# null 값 제외한 오류 값 : 252905
len(set(idx122) - set(idx123))

252905

In [94]:
df.iloc[:, 12].reindex(sorted(set(idx122) - set(idx123)))[:5]

C:\Users\kbj\AppData\Local\Temp\ipykernel_18336\1563131270.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df.iloc[:, 12].reindex(sorted(set(idx122) - set(idx123)))[:5]


5     VMY-KM-38
21    RMY-HD-20
28    KMY-HD-03
46    RMY-HD-20
62    MMY-HD-11
Name: 배출가스인증번호, dtype: object

### 13\. 배출가스등급
- 오류
    - null : 0 개

In [95]:
df.iloc[:, 13].value_counts()

4    1528586
Name: 배출가스등급, dtype: int64

In [96]:
df.iloc[:, 13].isnull().sum()

0

### 14\. [e]저공해조치종류
- 오류
    - null : 1,528,391 개

In [97]:
df.iloc[:, 14].value_counts()

2종 저감장치 부착    115
1종 저감장치 부착     71
엔진개조            4
3종 저감장치 부착      4
동시저감장치 부착       1
Name: 저공해조치종류, dtype: int64

In [98]:
df.iloc[:, 14].isnull().sum()

1528391

### 15\. 저공해조치구조변경YN
- 오류
    - null : 0 개

In [99]:
df.iloc[:, 15].value_counts()

N    1215924
Y     312662
Name: 저공해조치구조변경YN, dtype: int64

In [100]:
df.iloc[:, 15].isnull().sum()

0

### 16\. 차량말소YN
- 오류
    - null : 0 개

In [101]:
df.iloc[:, 16].value_counts()

N    1528586
Name: 차량말소YN, dtype: int64

In [102]:
df.iloc[:, 16].isnull().sum()

0

### 17\. [e]취득일자
- 오류
    - null :  93,072개
    - 8자리 아닌 값 : 2 개
    - 숫자가 아닌 값(nan값 제외) :  0 개

In [103]:
df.iloc[:, 17].value_counts()

20220607.0    1008
20220627.0     901
20220602.0     883
20220620.0     844
20220530.0     833
              ... 
20070221         1
20110306         1
20020628         1
20150415         1
20051218.0       1
Name: 취득일자, Length: 16929, dtype: int64

In [104]:
df.iloc[:, 17].isnull().sum()

93072

In [111]:
df['취득일자'].dtype

dtype('O')

In [113]:
idx171 = []
for i, one in enumerate(df['취득일자'].to_numpy()):
    try:
        int(one)
    except:
        idx171.append(i)

In [114]:
len(idx171)

93074

In [115]:
idx172 = df[df['취득일자'].isnull() == True]['취득일자'].index
len(idx172)

93072

In [116]:
df.iloc[sorted(set(idx171) - set(idx172)), 17]

735987     200109 9
1372039    2001 412
Name: 취득일자, dtype: object

### 18\. [e]취득금액
- 오류
    - null : 74,973 개
    - 숫자가 아닌 값 : 0 개

In [117]:
df.iloc[:, 18].value_counts()

0.0            258676
1000000.0       43125
500000.0        29818
2000000.0       28457
100000.0        27958
                ...  
62790.0             1
100761818.0         1
26188580.0          1
17523047.0          1
20240.0             1
Name: 취득금액, Length: 159946, dtype: int64

In [118]:
df.iloc[:, 18].isnull().sum()

74973

In [122]:
p18 = re.compile('\D+')
idx181 = []
idx182 = []
count = 0
for i, one in tqdm(enumerate(df.iloc[:, 18].to_numpy())):
    try:
        if p18.search(str(int(one))):
            idx181.append(i)
            count += 1
    except:
        idx182.append(i)
        count += 1
count

1528586it [00:02, 690456.77it/s]


74973

In [125]:
len(idx181), len(idx182)

(0, 74973)

In [123]:
df.iloc[:, 18].reindex(idx182)[:5]

C:\Users\kbj\AppData\Local\Temp\ipykernel_18336\2151798388.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df.iloc[:, 18].reindex(idx182)[:5]


62    NaN
98    NaN
128   NaN
146   NaN
156   NaN
Name: 취득금액, dtype: float64

### 19\. 차명
- 오류
    - null : 0 개

In [126]:
df.iloc[:, 19].value_counts()

포터Ⅱ(PORTERⅡ)              180795
싼타페(SANTAFE)              138850
스포티지                       76497
그랜드 스타렉스(GRAND STAREX)     74977
투싼(TUCSON)                 58687
                           ...  
CIVIC SiR                      1
TACOMA3.4                      1
미니브리티시오픈                       1
BMW 525itA                     1
메가트럭군물자수송                      1
Name: 차명, Length: 3572, dtype: int64

In [127]:
df.iloc[:, 19].isnull().sum()

0

### 20\. 차종_제원정보
- 오류
    - null : 0 개

In [128]:
df.iloc[:, 20].value_counts()

승용    843902
화물    521971
승합    155705
특수      7008
Name: 차종_제원정보, dtype: int64

In [129]:
df.iloc[:, 20].isnull().sum()

0

### 21\. 차종분류
- 오류
    - null : 0 개

In [130]:
df.iloc[:, 21].value_counts()

일반             540536
다목적            439052
일반형-카고         370138
밴               63114
특수용도형-특수용도형     51196
일반형-픽업          31406
기타              18027
덤프               5965
견인               5446
특수               1725
특수작업형             854
구난                708
승용겸화물             267
특수용도형-탱크로리        151
특수용도형-피견인           1
Name: 차종분류, dtype: int64

In [131]:
df.iloc[:, 21].isnull().sum()

0

### 22\. 차종유형
- 오류
    - null : 0 개

In [132]:
df.iloc[:, 22].value_counts()

중형    698798
소형    582232
대형    219531
경형     28025
Name: 차종유형, dtype: int64

In [134]:
df.iloc[:, 22].isnull().sum()

0

### 23\. [e]자동차형식
- 오류
    - null : 284 개

In [135]:
df.iloc[:, 23].value_counts()

CM7UBA               36235
CA26R                23887
JE553G-HG            22730
HRQJA-DX-B           19414
TQ-12JAB-1           16880
                     ...  
HD2-42-10Le-BEIT1        1
EN-7NS3DAF8-2            1
HBSV-VD138               1
KTH-18L14.5-410          1
HD050-8MT2-LGc-A         1
Name: 자동차형식, Length: 8114, dtype: int64

In [136]:
df.iloc[:, 23].isnull().sum()

284

### 24\. [e]제작사명
- 오류
    - null : 9,892 개

In [137]:
df.iloc[:, 24].value_counts()

현대자동차(주)       891178
기아자동차(주)       377792
대우자동차(주)        62506
한국지엠주식회사        53957
쌍용자동차(주)        41016
                ...  
(주) 미래교통렌트카         1
마이웨이                1
고랜드코리아              1
백상 CTC              1
서울시티투어버스(주)         1
Name: 제작사명, Length: 669, dtype: int64

In [138]:
df.iloc[:, 24].isnull().sum()

9892

### 25\. 연료
- 오류
    - null : 0 개

In [139]:
df.iloc[:, 25].value_counts()

경유             1167610
휘발유             334517
LPG(액화석유가스)      26419
기타연료                38
알코올                  1
CNG(압축천연가스)          1
Name: 연료, dtype: int64

In [140]:
df.iloc[:, 25].isnull().sum()

0

### 26\. [e]엔진형식
- 오류
    - null : 1,846 개

In [141]:
df.iloc[:, 26].value_counts()

D4CB           407006
D4EA           190781
J3             139727
D6EA            75036
Z20S            53958
                ...  
12A2DJ01            1
AR01544             1
LT-1                1
J3-8-VF-F           1
D2676 LOH02         1
Name: 엔진형식, Length: 1428, dtype: int64

In [142]:
df.iloc[:, 26].isnull().sum()

1846

### 27\. [e]총중량
- 오류
    - null : 1 개
    - 숫자가 아닌 값 : 0 개

In [143]:
df.iloc[:, 27].value_counts()

2925.0     66420
2285.0     53732
2950.0     51480
2955.0     47126
2345.0     39729
           ...  
6960.0         1
10570.0        1
35180.0        1
5435.0         1
21690.0        1
Name: 총중량, Length: 2121, dtype: int64

In [144]:
df.iloc[:, 27].isnull().sum()

1

In [145]:
p27 = re.compile('\D+')
idx271 = []
idx272 = []
count = 0
for i, one in tqdm(enumerate(df.iloc[:, 27].to_numpy())):
    try:
        if p27.search(str(int(one))):
            idx271.append(i)
            count += 1
    except:
        idx272.append(i)
        count += 1
count

1528586it [00:02, 662307.08it/s]


1

In [146]:
len(idx271), len(idx272)

(0, 1)

### 28\. [e]적재중량
- 오류
    - null : 511 개
    - 숫자가 아닌 값 : 0 개

In [149]:
df.iloc[:, 28].value_counts()

0.0         999646
1000.0      351435
600.0        41249
400.0        31382
800.0        24526
             ...  
9400.0           1
10200.0          1
450000.0         1
14200.0          1
800000.0         1
Name: 적재중량, Length: 164, dtype: int64

In [150]:
df.iloc[:, 28].isnull().sum()

511

In [151]:
p28 = re.compile('\D+')
idx281 = []
idx282 = []
count = 0
for i, one in tqdm(enumerate(df.iloc[:, 28].to_numpy())):
    try:
        if p28.search(str(int(one))):
            idx281.append(i)
            count += 1
    except:
        idx282.append(i)
        count += 1
count

1528586it [00:02, 690660.57it/s]


511

In [152]:
len(idx281), len(idx282)

(0, 511)

### 19\. [e]엔진출력
- 오류
    - null : 359 개
    - 숫자가 아닌 값 : 18 개

In [153]:
df.iloc[:, 29].value_counts()

126.0       289953
174.0       104789
151.0        91306
150.0        77553
126          64775
             ...  
280-6400         1
4                1
34.0             1
273.0            1
436.0            1
Name: 엔진출력, Length: 662, dtype: int64

In [154]:
df.iloc[:, 29].isnull().sum()

359

In [160]:
p29 = re.compile('\d+[.]\d+')
idx291 = []
idx292 = []
count = 0
for i, one in tqdm(enumerate(df.iloc[:, 29].to_numpy())):
    try:
        if not p29.search(str(float(one))):
            idx291.append(i)
            count += 1
    except:
        idx292.append(i)
        count += 1
count

1528586it [00:02, 708461.06it/s]


377

In [161]:
len(idx291), len(idx292)

(359, 18)

In [162]:
idx293 = df[df['엔진출력'].isnull() == True]['엔진출력'].index
len(idx293)

359

In [163]:
df.iloc[idx291, 29][:5]

C:\Users\kbj\AppData\Local\Temp\ipykernel_18336\2791330140.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df.iloc[idx291, 29][:5]


12060    NaN
16761    NaN
19514    NaN
22845    NaN
25508    NaN
Name: 엔진출력, dtype: object

In [164]:
df.iloc[idx292, 29][:5]

C:\Users\kbj\AppData\Local\Temp\ipykernel_18336\491642684.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df.iloc[idx292, 29][:5]


50704        114PS
114636       113ps
162707      0140PS
195884    105-5800
205207       195HP
Name: 엔진출력, dtype: object

In [167]:
df.iloc[idx293, 29][:5]

C:\Users\kbj\AppData\Local\Temp\ipykernel_18336\2073082367.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df.iloc[idx293, 29][:5]


12060    NaN
16761    NaN
19514    NaN
22845    NaN
25508    NaN
Name: 엔진출력, dtype: object

### 30\. [e]배기량
- error
    - null : 4
    - 숫자가 아닌 값 : 0

In [169]:
df.iloc[:, 30].value_counts()

2497.0     412725
1991.0     245078
2902.0     139727
2959.0      75053
1995.0      68705
            ...  
4701.0          1
3803.0          1
5958.0          1
15607.0         1
12412.0         1
Name: 배기량, Length: 627, dtype: int64

In [170]:
df.iloc[:, 30].isnull().sum()

4

In [171]:
p30 = re.compile('\D+')
idx301 = []
idx302 = []
count = 0
for i, one in tqdm(enumerate(df.iloc[:, 30].to_numpy())):
    try:
        if p30.search(str(int(one))):
            idx301.append(i)
            count += 1
    except:
        idx302.append(i)
        count += 1
count

1528586it [00:02, 619306.55it/s]


4

In [172]:
len(idx301), len(idx302)

(0, 4)

### 31\. 승차인원
- error
    - null : 0

In [173]:
df.iloc[:, 31].value_counts()

5     625751
3     338761
7     271132
6      92585
11     72915
12     58829
2      27413
9      20963
4       9575
25      4045
47      1446
34      1115
46       901
0        852
39       460
29       410
15       275
43       229
38       186
42       147
30       139
8         91
45        58
16        48
41        41
18        32
59        22
33        21
21        18
20        17
19        14
28        13
35        12
10         9
1          8
32         7
26         6
63         4
40         4
56         4
23         4
14         4
31         3
22         3
37         2
62         2
51         2
67         2
44         2
50         1
27         1
36         1
64         1
Name: 승차인원, dtype: int64

In [174]:
df.iloc[:, 31].isnull().sum()

0

In [180]:
p31 = re.compile('\D+')
idx311 = []
idx312 = []
count = 0
for i, one in tqdm(enumerate(df.iloc[:, 31].to_numpy())):
    try:
        if p31.search(str(int(one))):
            idx311.append(i)
            count += 1
    except:
        idx312.append(i)
        count += 1
count

1528586it [00:02, 756812.66it/s]


0

### 32\. [e]차량길이
- error
    - null : 1
    - 숫자 아닌 값 : 0

In [175]:
df.iloc[:, 32].value_counts()

4675.0     139634
5125.0     138323
5085.0     126095
5120.0      76767
4350.0      76260
            ...  
11970.0         1
4593.0          1
8285.0          1
5940.0          1
5435.0          1
Name: 차량길이, Length: 1247, dtype: int64

In [176]:
df.iloc[:, 32].isnull().sum()

1

In [177]:
p32 = re.compile('\D+')
idx321 = []
idx322 = []
count = 0
for i, one in tqdm(enumerate(df.iloc[:, 32].to_numpy())):
    try:
        if p32.search(str(int(one))):
            idx321.append(i)
            count += 1
    except:
        idx322.append(i)
        count += 1
count

1528586it [00:02, 701730.27it/s]


1

In [178]:
len(idx321), len(idx322)

(0, 1)

### 33\. [e]차량너비
- error
    - null : 1
    - 숫자 아닌 값 : 0

In [206]:
df.iloc[:, 33].value_counts()

1740.0    326413
1890.0    143036
1920.0    127426
1985.0     74746
1840.0     53628
           ...  
175.0          1
2465.0         1
1834.0         1
1633.0         1
1610.0         1
Name: 차량너비, Length: 318, dtype: int64

In [207]:
df.iloc[:, 33].isnull().sum()

1

In [208]:
p33 = re.compile('\D+')
idx331 = []
idx332 = []
count = 0
for i, one in tqdm(enumerate(df.iloc[:, 33].to_numpy())):
    try:
        if p33.search(str(int(one))):
            idx331.append(i)
            count += 1
    except:
        idx332.append(i)
        count += 1
count

1528586it [00:02, 685335.39it/s]


1

### 34\. [e]차량높이
- error
    - null : 3
    - 숫자 아닌 값 : 0

In [181]:
df.iloc[:, 34].value_counts()

1965.0    229617
1695.0     72455
1925.0     67094
1995.0     58511
1935.0     56907
           ...  
2210.0         1
1717.0         1
2615.0         1
1529.0         1
4000.0         1
Name: 차량높이, Length: 604, dtype: int64

In [182]:
df.iloc[:, 34].isnull().sum()

3

In [183]:
p34 = re.compile('\D+')
idx341 = []
idx342 = []
count = 0
for i, one in tqdm(enumerate(df.iloc[:, 34].to_numpy())):
    try:
        if p34.search(str(int(one))):
            idx341.append(i)
            count += 1
    except:
        idx342.append(i)
        count += 1
count

1528586it [00:02, 618882.59it/s]


3

### 35\. [e]원산지명
- error
    - null : 19,087
    - 문자 아닌 값 : 0

In [184]:
df.iloc[:, 35].value_counts()

대한민국           1467579
독일               16650
미국                7363
스웨덴               6916
프랑스               5140
영국                2990
일본                1476
네덜란드               676
오스트리아              298
이탈리아               209
기타                  78
캐나다                 31
벨기에                 27
멕시코                 24
대만                  13
중국                   9
가나                   5
스페인                  4
스위스                  3
괌                    2
덴마크                  2
기니비사우                1
감비아                  1
가봉                   1
독립국연합 (CIS)          1
Name: 원산지명, dtype: int64

In [185]:
df.iloc[:, 35].isnull().sum()

19087

In [187]:
p35 = re.compile('\D+')
idx351 = []
idx352 = []
count = 0
for i, one in tqdm(enumerate(df.iloc[:, 35].to_numpy())):
    try:
        if not p35.search(one):
            idx351.append(i)
            count += 1
    except:
        idx352.append(i)
        count += 1
count

1528586it [00:01, 1136214.58it/s]


19087

In [188]:
len(idx351), len(idx352)

(0, 19087)

### 36\. 삭제YN
- error
    - null : 0

In [189]:
df.iloc[:, 36].value_counts()

N    1528586
Name: 삭제YN, dtype: int64

In [190]:
df.iloc[:, 36].isnull().sum()

0

### 37\. [e]제원승인일자
- error
    - null : 431
    - 형식에 맞지 않는 값 : 108

In [191]:
df.iloc[:, 37].value_counts()

20100326      78288
20090622      35879
20100326.0    35819
20110524      29346
20090520      24716
              ...  
20160427          1
20151216          1
20121101          1
20050422          1
20070529.0        1
Name: 제원승인일자, Length: 5958, dtype: int64

In [192]:
df.iloc[:, 37].isnull().sum()

431

In [193]:
p37 = re.compile('\D+')
idx371 = []
idx372 = []
count = 0
for i, one in tqdm(enumerate(df.iloc[:, 37].to_numpy())):
    try:
        if p37.search(str(int(one))):
            idx371.append(i)
            count += 1
    except:
        idx372.append(i)
        count += 1
count

1528586it [00:02, 596189.99it/s]


539

In [194]:
len(idx371), len(idx372)

(0, 539)

In [195]:
df.iloc[:, 37].reindex(idx372)[:5]

C:\Users\kbj\AppData\Local\Temp\ipykernel_18336\829264895.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df.iloc[:, 37].reindex(idx372)[:5]


1544         NaN
3888         NaN
4231    1991 112
6957         NaN
7069         NaN
Name: 제원승인일자, dtype: object

In [196]:
idx373 = df[df['제원승인일자'].isnull() == True]['제원승인일자'].index
len(idx373)

431

In [197]:
len(list(set(idx372) - set(idx373)))

108

In [198]:
df.iloc[sorted(set(idx372) - set(idx373)), 37][:5]

C:\Users\kbj\AppData\Local\Temp\ipykernel_18336\4017740081.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df.iloc[sorted(set(idx372) - set(idx373)), 37][:5]


4231     1991 112
8926     1991 123
21579    1991 112
29975    1991 123
38288    1991 112
Name: 제원승인일자, dtype: object

### 38\. [e]구동형식
- error
    - null : 1,528,545

In [199]:
df.iloc[:, 38].value_counts()

4x2         18
6x2          9
10x4         5
후륜구동         4
4륜구동(선택)     1
4x4          1
기타           1
전륜구동         1
4륜구동(상시)     1
Name: 구동형식, dtype: int64

In [200]:
df.iloc[:, 38].isnull().sum()

1528545

### 39\. [e]변속기종류
- error
    - null : 160
    - 문자가 아닌 값 : 0

In [201]:
df.iloc[:, 39].value_counts()

자동        963870
수동        563865
기타           500
무단           158
반자동           32
변속기 없음         1
Name: 변속기종류, dtype: int64

In [202]:
df.iloc[:, 39].isnull().sum()

160

In [203]:
p39 = re.compile('[가-힣]+')
idx391 = []
idx392 = []
count = 0
for i, one in tqdm(enumerate(df.iloc[:, 39].to_numpy())):
    try:
        if not p39.search(one):
            idx391.append(i)
            count += 1
    except:
        idx392.append(i)
        count += 1
count

1528586it [00:01, 956223.37it/s]


160

In [204]:
len(idx391), len(idx392)

(0, 160)

# 코드 마지막